# Working with Distinct Values, $elemMatch, and Regex

## An exceptional laureate

In [ ]:
from pymongo import MongoClient

client = MongoClient()
db = client.nobel

In [ ]:
db.laureates.find_one({"prizes.2": {"$exists": True}})

## Using .distinct()

In [ ]:
db.laureates.distinct("gender")

- A convenience method for a common aggregation (like `count_documents`)
- We will not cover custom aggregations in this lesson, but the `aggregate` method is powerful.
- `distinct` aggregation is efficient if there is a collection *index* on the field
- We will learn how to create an index later in this lesson
- No index needed here: collection fits in memory, has ≲ 1,000 documents

## All prize categories vs those with multi-winners

In [ ]:
db.laureates.distinct("prizes.category")

In [ ]:
db.laureates.distinct(
  "prizes.category", {"prizes.1": {"$exists": True}})

## Enter $elemMatch

In [ ]:
db.laureates.count_documents({
    "prizes": {
        "category": "physics",
        "share": "1"
    }
})

In [ ]:
db.laureates.count_documents({
    "prizes.category": "physics",
    "prizes.share": "1"
})

In [ ]:
db.laureates.count_documents({
    "prizes": {
        "$elemMatch": {
            "category": "physics",
            "share": "1"
        }
    }
})

In [ ]:
db.laureates.count_documents({
    "prizes": {
        "$elemMatch": {
            "category": "physics",
            "share": "1",
            "year": {"$lt": "1945"},
        }
    }
})

## Finding a substring with $regex

In [ ]:
db.laureates.find_one({"firstname": "Marie"})

In [ ]:
case_sensitive = db.laureates.distinct(
    "bornCountry",
    {"bornCountry": {"$regex": "Poland"}})

In [ ]:
case_insensitive = db.laureates.distinct(
    "bornCountry",
    {"bornCountry": {"$regex": "poland", "$options": "i"}})

assert set(case_sensitive) == set(case_insensitive)

In [ ]:
from bson.regex import Regex

db.laureates.distinct("bornCountry", 
                      {"bornCountry": Regex("poland", "i")})

In [ ]:
import re

db.laureates.distinct("bornCountry",
                      {"bornCountry": re.compile("poland", re.I)})

## Beginning and ending (and escaping)

In [ ]:
from bson.regex import Regex

db.laureates.distinct("bornCountry", 
                      {"bornCountry": Regex("^Poland")})

In [ ]:
db.laureates.distinct(
    "bornCountry", 
     {"bornCountry": Regex("^Poland \(now")})

In [ ]:
db.laureates.distinct(
    "bornCountry", 
     {"bornCountry": Regex("now Poland\)$")})